In [10]:
from tweepy.streaming import StreamListener
from tweepy.auth import OAuthHandler
from tweepy import Stream

from datetime import datetime
import json

import pandas as pd
from pymongo import MongoClient

In [11]:
#pyenv
import os
from dotenv import load_dotenv
load_dotenv()
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')

In [12]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
#api = tweepy.API(auth)
#api.update_status('tweepy + oauth!')

In [37]:
client = MongoClient('localhost', 27017)
db = client.twitterdb
col = db.tweets
raw_col = db.raw_data
keywords = ['bieber']

In [38]:
class MyListener(StreamListener):
    def on_data(self,data):
        
        tweet = json.loads(data)
        created_at = tweet["created_at"]
        id_str = tweet[ "id_str"]
        text = tweet[ "text"]
        obj = {"created_at":created_at, "id_str":id_str, "text":text}
        print(obj)
        print('====================')
        tweetind = col.insert_one(obj).inserted_id
        print(data)
        tweetind_2 = raw_col.insert_many(tweet).inserted_id
        return True
    
    def on_error(self, status):
        print(status)
        return False

In [39]:
mylistener = MyListener()
mystream = Stream(auth, listener = mylistener)

In [40]:
print('start stream')
#mystream.filter(track=keywords)
mystream.filter(track = keywords)
print('close stream')

start stream
{'created_at': 'Fri May 03 18:47:37 +0000 2019', 'id_str': '1124385037464612864', 'text': 'RT @KennethxV_: going to have 257 songs https://t.co/dU4NQw4qXP'}
{"created_at":"Fri May 03 18:47:37 +0000 2019","id":1124385037464612864,"id_str":"1124385037464612864","text":"RT @KennethxV_: going to have 257 songs https:\/\/t.co\/dU4NQw4qXP","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":709104010,"id_str":"709104010","name":"_NICO_DATWAY","screen_name":"_nico_datway","location":null,"url":null,"description":"Lean into ur morning \ud83c\udf47","translator_type":"none","protected":false,"verified":false,"followers_count":176,"friends_count":185,"listed_count":0,"favourites_count":2910,"statuses_count":3509,"created_at":"Fri Oct 

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

In [ ]:
class LimitedListener(StreamListener):

    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 10

    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)

In [ ]:
limited_listener = LimitedListener()
limited_stream = Stream(auth, listener = limited_listener)
print('inicia limited stream')
limited_stream.filter(track=keywords)
print('fecha limited stream')

In [ ]:
"""
client = MongoClient('localhost', 27017)
db_test = client.testdb
collection_test = db_test.col_test
obj_test = {"putetinho": "ayn ayn"}
collection_test.insert_one(obj_test)
"""

In [ ]:
col.find_one()

In [ ]:
dataset = [{"created_at":item["created_at"],
           "text":item["text"],
           } for item in col.find()]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]